In [58]:
import pandas as pd
from pathlib import Path
import os

directory = Path(os.getcwd())   # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\HKUST-x-J.P.-Morgan-Fintech-Mentorship-Program\\model_testing\\supervised'
directory = directory.parent.parent.parent.parent.joinpath("data")  # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\data'
df = pd.read_pickle(directory.joinpath("stocktwits_processed_sample.pkl"))
df

,text,time,symbols,tokens
0,it's so easy to see insiders buying before th...,1639643526,$GOEV,easy see insiders buying big announcement
1,it's so easy to see insiders buying before th...,1639643526,$AAPL,easy see insiders buying big announcement
2,Santa's rally has an address to visit : “1...,1639643425,$QQQ,santas rally address visit “17000 ndx breakout...
3,Santa's rally has an address to visit : “1...,1639643425,$AAPL,santas rally address visit “17000 ndx breakout...
4,Santa's rally has an address to visit : “1...,1639643425,$SPY,santas rally address visit “17000 ndx breakout...
...,...,...,...,...
76591,suggested for US ARCA Major Markets - val...,1640145541,$XMI,suggested us arca major markets value screen
76592,and similar charts. We will see $180,1640143905,$SQ,similar charts see 180
76593,and similar charts. We will see $180,1640143905,$V,similar charts see 180
76594,damn putting in some work last couple of weeks..,1640139998,$V,damn putting work last couple weeks


In [3]:
sample = 500
# remove symbols with too little sample size
df = df.groupby('symbols').filter(lambda x : len(x)>sample)

# turn all symbols into the same sample size
i = sample
for j in set(df['symbols']):
    df = df[df['symbols'] != j].append(df[df['symbols'] == j].sample(i), ignore_index=True)
print(df['symbols'].value_counts())

$NIO     500
$JD      500
$TSLA    500
$GS      500
$MS      500
$JPM     500
$AAPL    500
$BAC     500
$AMZN    500
$PYPL    500
$AMD     500
$QQQ     500
$V       500
$MSFT    500
$SPY     500
$NVDA    500
$BABA    500
$FB      500
Name: symbols, dtype: int64


In [37]:
import datetime
df['time'] = [datetime.date.fromtimestamp(date) for date in df['time']]


TypeError: 'list' object cannot be interpreted as an integer

In [59]:
# remove symbols with too little sample size
df = df.groupby('symbols').filter(lambda x : len(x)>sample)

# turn all symbols into the same sample size
i = sample
for j in set(df['symbols']):
    df = df[df['symbols'] != j].append(df[df['symbols'] == j].sample(i), ignore_index=True)

In [60]:
df_test = pd.DataFrame(df[['text', 'symbols']])
df['time'] = [datetime.date.fromtimestamp(date) for date in df['time']]
df_test['text'] = [j[0]+j[1] for j in list(zip(list([str(i) for i in df['time']]), list(df['text'])))]
df_test

,text,symbols
0,2021-12-18,$NIO
1,2021-12-16 stupid bulls don't ask why there is...,$NIO
2,2021-12-20 the inflation/Covid panic games...,$NIO
3,2021-12-19 \n\n🔥🔥NIO VR GLASSES🔥🔥\n\n- Worlds ...,$NIO
4,"2021-12-22 Time it took to sell 100,000 cars:...",$NIO
...,...,...
8995,"2021-12-09 buy dec 17 OTM calls, hold through ...",$FB
8996,2021-12-16 360 by Christmas!,$FB
8997,2021-12-10 He will lead the world into a ...,$FB
8998,2021-12-08 Yall need to catch up to snap🚀🚀,$FB


In [61]:
# train_test split
df_train = df_test.sample(frac=0.8,random_state=200)
df_test = df_test.drop(df_train.index)

X_train = df_train.text
X_test = df_test.text

y_train = df_train.symbols
y_test = df_test.symbols

print("{} unique classes\t{} training samples\t{} test samples".format(len(set(y_train)), len(X_train), len(X_test)))

18 unique classes	7200 training samples	1800 test samples


In [62]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import NearMiss
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [66]:
model = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None))])


# Train the model using the training data
model.fit(X_train, y_train)
# Predict the symbols of the test data
y_pred = model.predict(X_test)
# accuracy
print("The accuracy is {}".format(accuracy_score(y_test, y_pred)))

The accuracy is 0.3372222222222222


In [ ]:
enumerate(list(df['time']), list(df['text']))

In [6]:
df_X = pd.DataFrame(df[['time', 'text']])
df_y = pd.DataFrame(df[['symbols']])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.20)
X_train

,time,text
2312,1639949599,AC/DC rocks but that's redundant
5777,1638966351,Needs to break $45 to rip it
616,1640187617,is the chosen china play by american funds. s...
3972,1640105065,\n\nBEARS HOWS THE SHORTING WORKING OUT ...
5696,1640134100,Tuesday Option Volumes\n\n 338k (66% call/34% ...
...,...,...
1399,1639523045,\n\n
8808,1639059761,215
271,1639511333,official bull run for all Chinese stocks star...
5242,1639988422,"circuit breakers, won't be surprised if we..."


In [9]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

In [21]:
X_train['time']

2312    1639949599
5777    1638966351
616     1640187617
3972    1640105065
5696    1640134100
           ...    
1399    1639523045
8808    1639059761
271     1639511333
5242    1639988422
8603    1638907218
Name: time, Length: 7200, dtype: object

https://towardsdatascience.com/how-to-combine-textual-and-numerical-features-for-machine-learning-in-python-dc1526ca94d9

Method 1: failed

In [25]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Create Function Transformer to use Feature Union
def get_numeric_data(x):
    return [record[:-2].astype(float) for record in x]

def get_text_data(x):
    return [record[-1] for record in x]

transfomer_numeric = FunctionTransformer(X_train['time'])
transformer_text = FunctionTransformer(X_train['text'])

# Create a pipeline to concatenate Tfidf Vector and Numeric data
# Use RandomForestClassifier as an example
pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(analyzer='word'))
            ]))
         ])),
    ('clf', RandomForestClassifier())
])

# Grid Search Parameters for RandomForest
param_grid = {'clf__n_estimators': np.linspace(1, 100, 10, dtype=int),
              'clf__min_samples_split': [3, 10],
              'clf__min_samples_leaf': [3],
              'clf__max_features': [7],
              'clf__max_depth': [None],
              'clf__criterion': ['gini'],
              'clf__bootstrap': [False]}

# Training config
kfold = StratifiedKFold(n_splits=7)
scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)
rf_model.fit(X_train, y_train)
rf_best = rf_model.best_estimator_

Fitting 7 folds for each of 20 candidates, totalling 140 fits


C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
140 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
140 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one

TypeError: 'Series' object is not callable

In [22]:

# Create Function Transformer to use Feature Union
def get_numeric_data(x):
    return [record[:-2].astype(float) for record in x]

def get_text_data(x):
    return [record[-1] for record in x]

transfomer_numeric = FunctionTransformer(X_train['time'])
transformer_text = FunctionTransformer(X_train['text'])

# Create a pipeline to concatenate Tfidf Vector and Numeric data
# Use RandomForestClassifier as an example
pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(analyzer='word'))
            ]))
         ])),
    ('clf', SVC())
])

# Grid Search Parameters for SGDClassifier
param_grid = {'clf__C': [0.1,1, 10, 100], 'clf__gamma': [1,0.1,0.01,0.001],'clf__kernel': ['rbf', 'poly', 'sigmoid']}
# Training config
kfold = StratifiedKFold(n_splits=7)

# Perform GridSearch
rf_model = GridSearchCV(pipeline, param_grid, refit=True, verbose=2)
rf_model.fit(X_train, y_train)
rf_best = rf_model.best_estimator_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END ..........clf__C=0.1, clf__gamma=1, clf__kernel=rbf; total time=   0.0s
[CV] END ..........clf__C=0.1, clf__gamma=1, clf__kernel=rbf; total time=   0.0s
[CV] END ..........clf__C=0.1, clf__gamma=1, clf__kernel=rbf; total time=   0.0s
[CV] END ..........clf__C=0.1, clf__gamma=1, clf__kernel=rbf; total time=   0.0s
[CV] END ..........clf__C=0.1, clf__gamma=1, clf__kernel=rbf; total time=   0.0s
[CV] END .........clf__C=0.1, clf__gamma=1, clf__kernel=poly; total time=   0.0s
[CV] END .........clf__C=0.1, clf__gamma=1, clf__kernel=poly; total time=   0.0s
[CV] END .........clf__C=0.1, clf__gamma=1, clf__kernel=poly; total time=   0.0s
[CV] END .........clf__C=0.1, clf__gamma=1, clf__kernel=poly; total time=   0.0s
[CV] END .........clf__C=0.1, clf__gamma=1, clf__kernel=poly; total time=   0.0s
[CV] END ......clf__C=0.1, clf__gamma=1, clf__kernel=sigmoid; total time=   0.0s
[CV] END ......clf__C=0.1, clf__gamma=1, clf__k

C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
240 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\samue\anaconda3\envs\jpm\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one

TypeError: 'Series' object is not callable

In [7]:
import pandas as pd
from pathlib import Path
import os

directory = Path(os.getcwd())   # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\HKUST-x-J.P.-Morgan-Fintech-Mentorship-Program\\model_testing\\supervised'
directory = directory.parent.parent.parent.parent.joinpath("data")  # 'c:\\Users\\samue\\OneDrive\\Desktop\\JPM Fintech\\data'
df = pd.read_pickle(directory.joinpath("stocktwits_processed_sample.pkl"))
df

,text,time,symbols,tokens
0,it's so easy to see insiders buying before th...,1639643526,$GOEV,easy see insiders buying big announcement
1,it's so easy to see insiders buying before th...,1639643526,$AAPL,easy see insiders buying big announcement
2,Santa's rally has an address to visit : “1...,1639643425,$QQQ,santas rally address visit “17000 ndx breakout...
3,Santa's rally has an address to visit : “1...,1639643425,$AAPL,santas rally address visit “17000 ndx breakout...
4,Santa's rally has an address to visit : “1...,1639643425,$SPY,santas rally address visit “17000 ndx breakout...
...,...,...,...,...
76591,suggested for US ARCA Major Markets - val...,1640145541,$XMI,suggested us arca major markets value screen
76592,and similar charts. We will see $180,1640143905,$SQ,similar charts see 180
76593,and similar charts. We will see $180,1640143905,$V,similar charts see 180
76594,damn putting in some work last couple of weeks..,1640139998,$V,damn putting work last couple weeks


In [10]:

df = df.sample(100)

for ind in range(len(df)):
    print(df.iloc[ind]['text'],'\t',df.iloc[ind]['symbols'])

 yall need to stfu and stop whining cuz ur option got crushed. Thats just how it gonna be. So quit playing options. Buy shares and hold. Enjoy ur retirement 👌 	 $PYPL
 agreed, after selling that much, it still held its ground even with the market 	 $TSLA
 was analyzed by 56 analysts. The buy consensus is at 83%. So analysts seem to be very confident about .  	 $PYPL
3 Reasons To Buy Facebook Stock Following Pullback  Also     	 $GOOGL
     	 $AABB
 lost against Mercedes for self-driving cars because of missing hardware (lidar).  has the best lidar system but was shorted down from $20 to $6.  might get back to $20 soon. 30 million shares short. Benchmark Luminar has a market cap. equal to $35 per  share. 
 
As the article shows, no CEO can define itself what is needed for self-driving cars. It will be defined only by regulators, e.g. here the German Federal Motor Transport Authority. 
 
Lidar delivers directly a 3D world, including depth. So, it reduces the needed computing power and in